In [68]:
import chess
import chess.pgn
import chess.syzygy

import matplotlib, seaborn
from matplotlib import pyplot as plt

from scipy import stats

In [69]:
def count_pieces(board, color=chess.WHITE, piece=chess.KNIGHT):
    # return number of given pieces
    return len(board.pieces(piece_type=piece, color=color))

In [70]:
def wg_w(board):
    ## return color of the player with rook and bishop
    if not (count_pieces(board, chess.WHITE, chess.PAWN) == 0 \
            and count_pieces(board, chess.WHITE, chess.KNIGHT) == 0 \
            and count_pieces(board, chess.WHITE, chess.QUEEN) == 0 \
            and count_pieces(board, chess.BLACK, chess.PAWN) == 0 \
            and count_pieces(board, chess.BLACK, chess.KNIGHT) == 0 \
            and count_pieces(board, chess.BLACK, chess.QUEEN) == 0):
        return False
    if count_pieces(board, chess.WHITE, chess.ROOK) == 1 \
            and count_pieces(board, chess.WHITE, chess.BISHOP) == 1 \
            and count_pieces(board, chess.BLACK, chess.ROOK) == 1 \
            and count_pieces(board, chess.BLACK, chess.BISHOP) == 0:
        return 'white'
    if count_pieces(board, chess.BLACK, chess.ROOK) == 1 \
            and count_pieces(board, chess.BLACK, chess.BISHOP) == 1 \
            and count_pieces(board, chess.WHITE, chess.ROOK) == 1 \
            and count_pieces(board, chess.WHITE, chess.BISHOP) == 0:
        return 'black'
    return False

In [72]:
def whos_winning(tb_result, board):
    x = tb_result * int(board.turn)
    if x < 0:
        return 'black'
    elif x > 0:
        return 'white'
    else:
        return 'draw'    

In [73]:
pgn_file = 'data/wg-w.pgn'
results = {}
total = 0

addit_moves = {}
with chess.syzygy.open_tablebase("data/syzygy") as tablebase:
    with open(pgn_file) as pgn:
        while 1:
            game = chess.pgn.read_game(pgn)
            if not game:
                break
            total += 1
            result = game.headers['Result']
            board = game.board()
            more_moves = 0
            for move in game.mainline_moves():
                board.push(move)
                color_wg = wg_w(board)
                if color_wg:
                    if more_moves == 0:
                        tb_result = tablebase.probe_wdl(board)
                        tb_color = whos_winning(tb_result, board)
                        results[result] = results.get(result, []) + [(color_wg, tb_result, board.turn, tb_color)]
                        more_moves += 1
                if more_moves > 0:
                    more_moves += 1
            addit_moves[(result, color_wg)] = addit_moves.get((result, color_wg), []) + [more_moves]
            #if more_moves > 150:
            #    print(more_moves, game.headers)

In [66]:
results['0-1']

[('black', 0, False, 'draw'),
 ('black', 0, False, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', -2, True, 'black'),
 ('black', 2, False, 'draw'),
 ('black', 2, False, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, False, 'draw'),
 ('black', 0, True, 'draw'),
 ('white', 0, False, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, False, 'draw'),
 ('black', -2, True, 'black'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 2, False, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('black', 0, True, 'draw'),
 ('white', 0, False, 'draw'),
 

In [43]:
len(results['1/2-1/2'])

814

In [44]:
len(results['1-0']), results['1-0'].count('white'), results['1-0'].count('black')

(307, 299, 8)

In [45]:
len(results['0-1']), results['0-1'].count('white'), results['0-1'].count('black')

(273, 12, 261)

In [47]:
814 + 307 + 273

1394

In [49]:
12/273 / (8/307)

1.6868131868131868